In [1]:
import csv
import cv2
import torch
import numpy as np
from pathlib import Path
from datetime import datetime
from boxmot import DeepOCSORT

model = torch.hub.load('C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\yolov7-main\\yolov7-main', 'custom', "C:\\Users\\jithi\\Downloads\\yolov7.pt",force_reload=True, source='local',trust_repo=True)
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),
    device='cuda:0',
    fp16=True,
)

c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]   

Adding autoShape... 


2023-11-18 18:29:13.340 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:207 - Successfully loaded pretrained weights from "osnet_x0_25_msmt17.pt"
2023-11-18 18:29:13.342 | WARNING  | boxmot.appearance.reid_model_factory:load_pretrained_weights:211 - The following layers are discarded due to unmatched keys or layer size: ('classifier.weight', 'classifier.bias')


In [19]:
import gradio as gr
import json
import cv2
from datetime import datetime

# Initialize checkbox states
checkbox_states = {"person": False, "cat": False, "car": False, "truck": False,"bicycle":False,"motorcycle":False,"dog":False}

def update_checkboxes_and_process_video(person, cat, car, truck, bicycle, motorcycle, dog, video_path):
    # Update checkbox states
    checkbox_states["person"] = person
    checkbox_states["cat"] = cat
    checkbox_states["car"] = car
    checkbox_states["truck"] = truck
    checkbox_states["bicycle"] = bicycle
    checkbox_states["motorcycle"] = motorcycle
    checkbox_states["dog"] = dog
    checkbox_states["path"] = video_path
    # Write to JSON file
    with open("checkbox_states.json", "w") as f:
        json.dump(checkbox_states, f)

    # Process Video
    if video_path:
        process_video(video_path)  # Function to process the video, similar to your first code snippet

    return f"Updated and saved checkbox states: {checkbox_states}"

def process_video(video_path):
    # video_path="C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
    json_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\checkbox_states.json"
    
    with open(json_path, 'r') as f:
            checkbox_states = json.load(f)
    video_path=checkbox_states.get("path")
    cap = cv2.VideoCapture(video_path)
    csv_file = open('logs.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Time', 'ID', 'Class'])

    # Define colors for each class
    class_colors = {
        0: (0, 0, 255),
        2: (0, 255, 0),
        16: (255, 0, 0),
        # Add more classes and their colors
    }
    id_to_name = {
    0: 'person',
    2: 'car',
    16: 'dog'
    # 16:'none',
    # 7:'bird',
    # 58:'pot',
    # 10:'none'
    }

    thickness = 2
    fontscale = 0.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        with open(json_path, 'r') as f:
            checkbox_states = json.load(f)
        results = model(frame)
        df = results.pandas().xyxy[0]
        detections = []

            # Check if this class should have a bounding box drawn, according to the JSON file
        for _, row in df.iterrows():
            class_name = row['class']
            class_str=id_to_name.get(class_name)
            
            # x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            # class_name = row['class']
            # detections.append([x1, y1, x2, y2, 1.0, class_name])
                
                
            if checkbox_states.get(str(class_str), False):
                x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                class_name = row['class']
                detections.append([x1, y1, x2, y2, 1.0, class_name])

        tracks = tracker.update(np.array(detections), frame)

        xyxys = tracks[:, 0:4].astype('int')
        ids = tracks[:, 4].astype('int')
        confs = tracks[:, 5]
        clss = tracks[:, 6].astype('int')
        inds = tracks[:, 7].astype('int')

        if tracks.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
                color = class_colors.get(cls, (0, 0, 0))  # Default color is black
                frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
                text = f'id: {id}, class: {id_to_name.get(cls)}'
                frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                csv_writer.writerow([current_time, id, cls])    

        cv2.imshow('Video with Object IDs and Classes', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Gradio interface setup
demo = gr.Interface(
    update_checkboxes_and_process_video,
    [
        gr.Checkbox(initial_value=checkbox_states["person"], label="person"),
        gr.Checkbox(initial_value=checkbox_states["cat"], label="cat"),
        gr.Checkbox(initial_value=checkbox_states["car"], label="car"),
        gr.Checkbox(initial_value=checkbox_states["truck"], label="truck"),
        gr.Checkbox(initial_value=checkbox_states["bicycle"], label="bicycle"),
        gr.Checkbox(initial_value=checkbox_states["motorcycle"], label="motorcycle"),
        gr.Checkbox(initial_value=checkbox_states["dog"], label="dog"),
        gr.Textbox(label="Video Path"),
    ],
    "text"
)

if __name__ == "__main__":
    demo.launch()


C:\Users\jithi\AppData\Local\Temp\ipykernel_19512\1359734425.py:115: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states["person"], label="person"),
C:\Users\jithi\AppData\Local\Temp\ipykernel_19512\1359734425.py:116: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states["cat"], label="cat"),
C:\Users\jithi\AppData\Local\Temp\ipykernel_19512\1359734425.py:117: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states["car"], label="car"),
C:\Users\jithi\AppData\Local\Temp\ipykernel_19512\1359734425.py:118: GradioUnusedKwargWarning: You have unused kwarg parameters in Checkbox, please remove them: {'initial_value': False}
  gr.Checkbox(initial_value=checkbox_states["truck"]

Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


In [2]:
import gradio as gr
import json
import cv2
from datetime import datetime
import pandas as pd

id_to_name = {
    0: 'person',
    2: 'car',
    16: 'dog'
    # 16:'none',
    # 7:'bird',
    # 58:'pot',
    # 10:'none'
    }


# Initialize checkbox states
checkbox_states = {"person": False, "cat": False, "car": False, "truck": False,"bicycle":False,"motorcycle":False,"dog":False}

def create_entry_exit_logs():
    file_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\logs.csv"

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert the 'Time' column to datetime format for accurate processing
    df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y-%m-%d %H:%M:%S')

    # Group by ID and get the first and last time for each ID
    result_df = df.groupby('ID').agg(
        Class=('Class', 'first'),
        Entry_Time=('Time', 'first'),
        Exit_Time=('Time', 'last')
    ).reset_index()    
    result_df['Class'] = result_df['Class'].map(id_to_name)

    return result_df

def update_checkboxes_and_process_video(person, cat, car, truck, bicycle, motorcycle, dog, video_path):
    # Update checkbox states
    checkbox_states["person"] = person
    checkbox_states["cat"] = cat
    checkbox_states["car"] = car
    checkbox_states["truck"] = truck
    checkbox_states["bicycle"] = bicycle
    checkbox_states["motorcycle"] = motorcycle
    checkbox_states["dog"] = dog
    checkbox_states["path"] = video_path
    # Write to JSON file
    with open("checkbox_states.json", "w") as f:
        json.dump(checkbox_states, f)

    # Process Video
    if video_path:
        process_video(video_path)  # Function to process the video, similar to your first code snippet

    return f"Updated and saved checkbox states: {checkbox_states}"

def process_video(video_path):
    # video_path="C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
    json_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\checkbox_states.json"
    
    with open(json_path, 'r') as f:
            checkbox_states = json.load(f)
    video_path=checkbox_states.get("path")
    cap = cv2.VideoCapture(video_path)
    csv_file = open('logs.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Time', 'ID', 'Class'])

    # Define colors for each class
    class_colors = {
        0: (0, 0, 255),
        2: (0, 255, 0),
        16: (255, 0, 0),
        # Add more classes and their colors
    }
    # id_to_name = {
    # 0: 'person',
    # 2: 'car',
    # 16: 'dog'
    # # 16:'none',
    # # 7:'bird',
    # # 58:'pot',
    # # 10:'none'
    # }

    thickness = 2
    fontscale = 0.5

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        with open(json_path, 'r') as f:
            checkbox_states = json.load(f)
        results = model(frame)
        df = results.pandas().xyxy[0]
        detections = []

            # Check if this class should have a bounding box drawn, according to the JSON file
        for _, row in df.iterrows():
            class_name = row['class']
            class_str=id_to_name.get(class_name)

            if checkbox_states.get(str(class_str), False):
                x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                class_name = row['class']
                detections.append([x1, y1, x2, y2, 1.0, class_name])

        tracks = tracker.update(np.array(detections), frame)

        xyxys = tracks[:, 0:4].astype('int')
        ids = tracks[:, 4].astype('int')
        confs = tracks[:, 5]
        clss = tracks[:, 6].astype('int')
        inds = tracks[:, 7].astype('int')

        if tracks.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
                color = class_colors.get(cls, (0, 0, 0))  # Default color is black
                frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
                text = f'id: {id}, class: {id_to_name.get(cls)}'
                frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                csv_writer.writerow([current_time, id, cls])    

        cv2.imshow('Video with Object IDs and Classes', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='font-size: 40px;'>SentinelGuard</h1>")
    with gr.Tabs():
        with gr.TabItem("Parameters"):
            with gr.Row():
                checkbox_person = gr.Checkbox(label="person", value=checkbox_states["person"])
                checkbox_cat = gr.Checkbox(label="cat", value=checkbox_states["cat"])
                checkbox_car = gr.Checkbox(label="car", value=checkbox_states["car"])
                checkbox_truck = gr.Checkbox(label="truck", value=checkbox_states["truck"])
                checkbox_bicycle = gr.Checkbox(label="bicycle", value=checkbox_states["bicycle"])
                checkbox_motorcycle = gr.Checkbox(label="motorcycle", value=checkbox_states["motorcycle"])
                checkbox_dog = gr.Checkbox(label="dog", value=checkbox_states["dog"])
            video_path_input = gr.Textbox(label="Video Path")
            submit_button = gr.Button("Submit")
            submit_button.click(
                update_checkboxes_and_process_video, 
                inputs=[checkbox_person, checkbox_cat, checkbox_car, checkbox_truck, checkbox_bicycle, checkbox_motorcycle, checkbox_dog, video_path_input], 
                outputs=[]
            )
        with gr.TabItem("Logs"):
            create_logs_button = gr.Button("Create Entry/Exit time logs")
            logs_output = gr.Dataframe()
            create_logs_button.click(
                create_entry_exit_logs,
                outputs=logs_output
            )
if __name__ == "__main__":
    demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [2]:
import gradio as gr
import json
import cv2
from datetime import datetime
import pandas as pd

id_to_name = {
    0: 'person',
    2: 'car',
    16: 'dog'
    # 16:'none',
    # 7:'bird',
    # 58:'pot',
    # 10:'none'
    }


# Initialize checkbox states
checkbox_states = {"person": False, "cat": False, "car": False, "truck": False,"bicycle":False,"motorcycle":False,"dog":False}

def create_entry_exit_logs():
    file_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\logs.csv"

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Convert the 'Time' column to datetime format for accurate processing
    df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y-%m-%d %H:%M:%S')

    # Group by ID and get the first and last time for each ID
    result_df = df.groupby('ID').agg(
        Class=('Class', 'first'),
        Entry_Time=('Time', 'first'),
        Exit_Time=('Time', 'last'),
        Vid_Timestamp=('vid_timestamp', 'first')  # Include Vid_Timestamp
    ).reset_index()   
    result_df['Class'] = result_df['Class'].map(id_to_name)

    return result_df

def update_checkboxes_and_process_video(person, cat, car, truck, bicycle, motorcycle, dog, video_path):
    # Update checkbox states
    checkbox_states["person"] = person
    checkbox_states["cat"] = cat
    checkbox_states["car"] = car
    checkbox_states["truck"] = truck
    checkbox_states["bicycle"] = bicycle
    checkbox_states["motorcycle"] = motorcycle
    checkbox_states["dog"] = dog
    checkbox_states["path"] = video_path
    # Write to JSON file
    with open("checkbox_states.json", "w") as f:
        json.dump(checkbox_states, f)

    # Process Video
    if video_path:
        process_video(video_path)  # Function to process the video, similar to your first code snippet

    return f"Updated and saved checkbox states: {checkbox_states}"

def process_video(video_path):
    # video_path="C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\track.mp4"
    json_path = "C:\\Users\\jithi\\OneDrive\\Desktop\\VsCode\\capstone\\checkbox_states.json"
    
    with open(json_path, 'r') as f:
            checkbox_states = json.load(f)
    video_path=checkbox_states.get("path")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    csv_file = open('logs.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Time', 'ID', 'Class','vid_timestamp'])

    # Define colors for each class
    class_colors = {
        0: (0, 0, 255),
        2: (0, 255, 0),
        16: (255, 0, 0),
        # Add more classes and their colors
    }
    # id_to_name = {
    # 0: 'person',
    # 2: 'car',
    # 16: 'dog'
    # # 16:'none',
    # # 7:'bird',
    # # 58:'pot',
    # # 10:'none'
    # }

    thickness = 2
    fontscale = 0.5
    current_frame = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        start_time = cv2.getTickCount()
        with open(json_path, 'r') as f:
            checkbox_states = json.load(f)
        results = model(frame)
        df = results.pandas().xyxy[0]
        detections = []

            # Check if this class should have a bounding box drawn, according to the JSON file
        for _, row in df.iterrows():
            class_name = row['class']
            class_str=id_to_name.get(class_name)

            if checkbox_states.get(str(class_str), False):
                x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                class_name = row['class']
                detections.append([x1, y1, x2, y2, 1.0, class_name])

        tracks = tracker.update(np.array(detections), frame)

        xyxys = tracks[:, 0:4].astype('int')
        ids = tracks[:, 4].astype('int')
        confs = tracks[:, 5]
        clss = tracks[:, 6].astype('int')
        inds = tracks[:, 7].astype('int')
        if tracks.shape[0] != 0:
            for xyxy, id, conf, cls in zip(xyxys, ids, confs, clss):
                color = class_colors.get(cls, (0, 0, 0))  # Default color is black
                frame = cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), color, thickness)
                text = f'id: {id}, class: {id_to_name.get(cls)}'
                frame = cv2.putText(frame, text, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, fontscale, color, thickness)
                end_time = cv2.getTickCount()
                processing_time = (end_time - start_time) / cv2.getTickFrequency()
                vid_timestamp = current_frame / fps + processing_time

                current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                csv_writer.writerow([current_time, id, cls,vid_timestamp])    
                current_frame += 1

        cv2.imshow('Video with Object IDs and Classes', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='font-size: 40px;'>SentinelGuard</h1>")
    with gr.Tabs():
        with gr.TabItem("Parameters"):
            with gr.Row():
                checkbox_person = gr.Checkbox(label="person", value=checkbox_states["person"])
                checkbox_cat = gr.Checkbox(label="cat", value=checkbox_states["cat"])
                checkbox_car = gr.Checkbox(label="car", value=checkbox_states["car"])
                checkbox_truck = gr.Checkbox(label="truck", value=checkbox_states["truck"])
                checkbox_bicycle = gr.Checkbox(label="bicycle", value=checkbox_states["bicycle"])
                checkbox_motorcycle = gr.Checkbox(label="motorcycle", value=checkbox_states["motorcycle"])
                checkbox_dog = gr.Checkbox(label="dog", value=checkbox_states["dog"])
            video_path_input = gr.Textbox(label="Video Path")
            submit_button = gr.Button("Submit")
            submit_button.click(
                update_checkboxes_and_process_video, 
                inputs=[checkbox_person, checkbox_cat, checkbox_car, checkbox_truck, checkbox_bicycle, checkbox_motorcycle, checkbox_dog, video_path_input], 
                outputs=[]
            )
        with gr.TabItem("Logs"):
            create_logs_button = gr.Button("Create Entry/Exit time logs")
            logs_output = gr.Dataframe()
            create_logs_button.click(
                create_entry_exit_logs,
                outputs=logs_output
            )
if __name__ == "__main__":
    demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Traceback (most recent call last):
  File "c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\gradio\routes.py", line 516, in predict
    output = await route_utils.call_process_api(
  File "c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\gradio\route_utils.py", line 219, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\gradio\blocks.py", line 1437, in process_api
    result = await self.call_function(
  File "c:\Users\jithi\.conda\envs\pytorch_OIIO\lib\site-packages\gradio\blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(